## Code to reproduce Jensen-Shannon distance bug in Evidently

https://github.com/evidentlyai/evidently/issues/494

also a bug here:
https://github.com/microsoft/SynapseML/issues/2006

### Describe the problem

Your [current implementation for the Jensen-Shannon Distance (JSD)](https://github.com/evidentlyai/evidently/blob/main/src/evidently/calculations/stattests/jensenshannon.py) is leveraging [scipy.spatial.distance.jensenshannon](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jensenshannon.html), which defaults to computing the JSD using logarithms with base e. Under this definition, the JSD is bound between 0 and sqrt(ln(2))=0.83255... (see, for example, the [JSD Wikipedia article](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence))

I believe you want to compute JSD using base 2 logarithms, so that it is bound between 0 and 1 as you state in your [data drift detection blog post](https://www.evidentlyai.com/blog/data-drift-detection-large-datasets).

Here is code which reproduces this issue by computing the JSD between two extremely drifted distributions: 
```
from evidently.calculations.stattests import jensenshannon_stat_test
import numpy as np
import pandas as pd

x = pd.Series(np.random.normal(100, 10, 100_000))
y = pd.Series(np.random.normal(10_000, 10, 100_000))

print(jensenshannon_stat_test(x, y, feature_type='num',threshold=0.1))
>>> StatTestResult(drift_score=0.8325546111576977, drifted=True, actual_threshold=0.1)
```
Note that the `drift_score` is close to the current maximum theoretical value of `sqrt(ln(2))=0.83255...`, instead of the desired value of `1`. 

In [4]:
from evidently.calculations.stattests import jensenshannon_stat_test
import numpy as np
import pandas as pd

# Create two distributions
x = pd.Series(np.random.normal(100, 10, 100_000))
y = pd.Series(np.random.normal(10_000, 10, 100_000))


# Calculate the Jensen-Shannon divergence
print(jensenshannon_stat_test(x, y, feature_type='num',threshold=0.1))

StatTestResult(drift_score=0.8325546111576977, drifted=True, actual_threshold=0.1)


In [11]:
from scipy.spatial import distance
from utils import compute_optimal_histogram_bin_edges

bin_edges = compute_optimal_histogram_bin_edges(x, y, bin_strategy='stu')
x_percent = np.histogram(x, bins=bin_edges)[0] / len(x)
y_percent = np.histogram(y, bins=bin_edges)[0] / len(y)


print(f'Jensen-Shannon Distance base-e entropy (defalut): {distance.jensenshannon(x_percent, y_percent)}')

print(f'Jensen-Shannon Distance base-2 entropy: {distance.jensenshannon(x_percent, y_percent, base=2)}')


Jensen-Shannon Distance base-e entropy (defalut): 0.8325546111576977
Jensen-Shannon Distance base-2 entropy: 1.0
